# Speech emotion classification

In [1]:
import numpy as np
import pandas as pd
from pandas import Series
from pandas import concat
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from numpy import concatenate

Using TensorFlow backend.
/Users/mimi/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/mimi/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/mimi/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/mimi/anaconda3/lib/python3.7/site-packages/tensorflo

In [2]:
# create label by shifting 1 row
def create_label(df):
    df['id'] = df['_attribute_name_string'].astype(str).str[:2]
    #shift time label
    df['label'] = df.groupby(['id'])['emotion'].shift(-1)
    return df

# create timesteps
def get_sup(data, timesteps):
    # transform data to be supervised learning
    df = pd.DataFrame(data.values)
    columns = [df.shift(-i) for i in range(1, timesteps+1)]
    
    #convert list into dataframe
    dataframe = concat(columns, axis=1)
    #merge with the initial column
    result = concat([df, dataframe], axis=1, sort=False)
    supervised_values = result.values[:-timesteps,:]
    return supervised_values 

# select features in dataframe
def select_features(df, start_cols, end_cols, timesteps):
    lst = []
    for column in df.columns[start_cols:end_cols]:
        lst.append(get_sup(df[column], timesteps))
    return lst

#reformat into timesteps and remove nan, the more timestep, the more data is removed.
def re_format(y,x):
    a = np.array(y)    
    b = np.array(x).transpose([1,0,2])
    
    def no_nan(list_a):
        return all(x == x for x in list_a) # oppsite of any(x!=x)
    filtered = [no_nan(x) for x in a]
    # filter only x and y with ~nan  
    y = np.array([y[-1] for y in a[filtered]])
    Xs  = b[filtered]
    return y, Xs


 # fit an LSTM network to training data
def fit_lstm(X, y, batch_size, nb_epoch, neurons):
    model = Sequential()
    print(f"time")
    print(X.shape[1],  X.shape[2])
    model.add(LSTM(neurons, batch_input_shape=(
        batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    for i in range(nb_epoch):
        # if don't want to show training stages, set verbose=0
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.reset_states()
    return model

# validate data by average precision_score, can be changed depends on use cases
def predictor(model, batch_size, X, y):
    yhat = model.predict(X, batch_size=batch_size)
    average_precision = average_precision_score(y, yhat)
    return average_precision

In [3]:
def main():
    
    # for demo, I selected only 4 features 
    start_cols = 2
    end_cols = 6

    #model hyperparameters
    timesteps = 2 # starts from 0
    batch_size = 2
    epochs = 100
    neurons = 2
    
    df = pd.read_csv('speech_extracted_sample.csv')
    # data preprocessing
    prep_df = create_label(df)
    y1 = get_sup(prep_df['label'], timesteps)
    x1s = select_features(prep_df, start_cols, end_cols, timesteps)
    y_all, x_all = re_format(y1, x1s)
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.3, random_state=42)
    # model fitting
    model = fit_lstm(X_train, y_train, batch_size, epochs, neurons)
    # forecasting
    average_precision = predictor(model, batch_size, X_test, y_test)
    print(f"average precision score: {average_precision}")

In [4]:
# for demo, I selected only 4 features 
start_cols = 2
end_cols = 6

#model hyperparameters
timesteps = 2  # start from 0
batch_size = 2
epochs = 50
neurons = 2

df = pd.read_csv('data_test.csv')
# data preprocessing
prep_df = create_label(df)
y1 = get_sup(prep_df['label'], timesteps)
x1s = select_features(prep_df, start_cols, end_cols, timesteps)
#remove nan and select y
y_all, x_all = re_format(y1, x1s)
# train test split
X_train, X_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.3, random_state=42)
# model fitting
model = fit_lstm(X_train, y_train, batch_size, epochs, neurons)
# forecasting
average_precision = predictor(model, batch_size, X_test, y_test)
print(f"average precision score: {average_precision}")

time
4 3
Epoch 1/1
154/154 [==============================] - 0s 3ms/step - loss: 0.6108
Epoch 1/1
154/154 [==============================] - 0s 859us/step - loss: 0.4524
Epoch 1/1
154/154 [==============================] - 0s 831us/step - loss: 0.3360
Epoch 1/1
154/154 [==============================] - 0s 847us/step - loss: 0.2734
Epoch 1/1
154/154 [==============================] - 0s 846us/step - loss: 0.2280
Epoch 1/1
154/154 [==============================] - 0s 827us/step - loss: 0.1915
Epoch 1/1
154/154 [==============================] - 0s 922us/step - loss: 0.1666
Epoch 1/1
154/154 [==============================] - 0s 842us/step - loss: 0.1500
Epoch 1/1
154/154 [==============================] - 0s 847us/step - loss: 0.1384
Epoch 1/1
154/154 [==============================] - 0s 836us/step - loss: 0.1300
Epoch 1/1
154/154 [==============================] - 0s 834us/step - loss: 0.1237
Epoch 1/1
154/154 [==============================] - 0s 841us/step - loss: 0.1189
Epoch 1/1